In [1]:
# Speed comparison SH
import pandas as pd
import numpy as np
import glob
import os

path = np.array(glob.glob('/mnt/data/20201230/zt_7401_20201230_10.235.16.169_postvde.tar.gz/Logs_full/mdLog_SH_***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20201230/logs_20201230_zt_88_03_day_88data.tar.gz/mdLog_SH_***'))
assert(len(path1) == 1)
print(path1[0])

logSH = pd.read_csv(path1[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
logSH1 = pd.read_csv(path[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
fasterPerc = []
fasterTm25 = []
fasterTm = []
fasterTm75 = []
slowerTm25 = []
slowerTm = []
slowerTm75 = []

display(logSH['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(logSH1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())

/mnt/data/20201230/zt_7401_20201230_10.235.16.169_postvde.tar.gz/Logs_full/mdLog_SH_20201230_0855.csv
/mnt/data/20201230/logs_20201230_zt_88_03_day_88data.tar.gz/mdLog_SH_20201230_0834.csv
original data source type: 
[22 23]
new data source type: 
[22 23]
['08:34:49.790' '08:34:52.130' '08:34:52.180' ... '15:11:39.730'
 '15:11:39.000' '15:11:40.900']
['08:55:58.060' '08:55:59.930' '08:55:59.000' ... '15:59:58.080'
 '16:00:00.620' '16:00:02.010']


array([1, 2, 5, 6, 7, 4])

array([1, 2, 3, 5, 6, 7, 4])

In [2]:
### set here ***
columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]

num = 2
### set here ***
    
if num == 2:
    for i in range(0, num):
        in_dex = [16, 300, 852, 905]
        data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & \
                      (logSH['source'] == np.sort(logSH['source'].unique())[i])]
        data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == np.sort(logSH1['source'].unique())[i])]
        data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
        data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
        n1 = len(data1_1["StockID"].unique())
        n2 = len(data2_1["StockID"].unique())

        print(n1)
        print(n2)
        if n1 != n2:
            sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
            data1_1 = data1_1[data1_1["StockID"].isin(sl)]
            data2_1 = data2_1[data2_1["StockID"].isin(sl)]
        for cols in ['cum_amount']:
            data1_1[cols] = data1_1[cols].round(2)
            data2_1[cols] = data2_1[cols].round(2)
        data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
        test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
        n1 = test["sequenceNo_x"].count()
        n2 = test["sequenceNo_y"].count()
        len1 = len(test)
        print(n1)
        print(n2)
        print(len1)
        print("-----------------------------------------------")
        if n2 < len1:
            display("test is not complete:")
            display(test[np.isnan(test["sequenceNo_y"])])
            print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
        if (len1 == n2) & (n1 < len1):
            display("baseline is not complete:")
            display(test[np.isnan(test["sequenceNo_x"])])
            display(n2-n1)
            print((n2-n1)/n1)

else:
    in_dex = [16, 300, 852, 905]
    data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == logSH['source'].max())]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == logSH1['source'].max())]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())

    print(n1)
    print(n2)
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for cols in ['cum_amount']:
        data1_1[cols] = data1_1[cols].round(2)
        data2_1[cols] = data2_1[cols].round(2)
    data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["sequenceNo_x"])])
        display(n2-n1)
        print((n2-n1)/n1)  
    

in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]

display(data1['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())
display(data2['cum_amount'].astype(str).apply(lambda x: len(x.split('.')[1])).unique())


### set here ***
columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
### set here ***

data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])


1770
1459
4569230
4569230
4569230
-----------------------------------------------
1770
1457
4851760
4851760
4851760
-----------------------------------------------


array([1, 5, 4])

array([1, 5, 4])

11415
11415
11415


In [6]:
pd.set_option('max_columns', 200)
logSH[(logSH['StockID'] == 600769) & (logSH['source'] == 22) & (logSH['cum_volume'] == 32300)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
410010,1609118702650295,1718271,22,600769,SH,92500710,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
417048,1609118716771587,1929103,22,600769,SH,92514820,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
419938,1609118746773897,1945474,22,600769,SH,92544690,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
422654,1609118776778590,1950517,22,600769,SH,92614690,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
425272,1609118806786476,1953135,22,600769,SH,92644700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
427988,1609118836793633,1958178,22,600769,SH,92714700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
430606,1609118866798379,1960796,22,600769,SH,92744700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
433322,1609118896799547,1965839,22,600769,SH,92814810,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
435940,1609118926806798,1968457,22,600769,SH,92844710,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
438656,1609118956810442,1973500,22,600769,SH,92914700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17


In [7]:
logSH1[(logSH1['StockID'] == 600769) & (logSH1['source'] == 22) & (logSH1['cum_volume'] == 32300)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
39813,1609118682243202,49817,22,600769,SH,92500710,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
45650,1609118696364015,169342,22,600769,SH,92514820,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
48080,1609118726366343,185142,22,600769,SH,92544690,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
50333,1609118756372080,187395,22,600769,SH,92614690,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
52497,1609118786379120,189559,22,600769,SH,92644700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
54750,1609118816386635,191812,22,600769,SH,92714700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
56914,1609118846390004,193976,22,600769,SH,92744700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
59166,1609118876390215,196228,22,600769,SH,92814810,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
61326,1609118906396909,198388,22,600769,SH,92844710,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17
63575,1609118936401413,200637,22,600769,SH,92914700,32300,176358.0,5.46,5.45,5.44,5.43,5.42,5.41,10400,7900,22900,16400,127800,5.46,5.49,5.5,5.51,5.53,12300,1500,600,2700,2000,5.46,17


In [44]:
pd.set_option('max_columns', 200)
columns = ["StockID", "time", "cum_volume", "close", "bid1p", "bid2p", "bid1q", "bid2q",
           "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]
logSH1[(logSH1['StockID'] == 600295) & (logSH1['time'] >= 145657000)][columns].head()

,StockID,time,cum_volume,close,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
5021866,600295,145657000,23669495,10.50,10.47,10.46,3000,4600,10.49,10.5,10.51,10.52,10.53,10400,19500,9500,28300,6800,10.65,26996
5023883,600295,145700000,23669995,10.49,0.00,0.00,0,0,0.00,0.0,0.00,0.00,0.00,0,0,0,0,0,10.65,26998
5023885,600295,145700000,23669995,10.49,0.00,0.00,0,0,0.00,0.0,0.00,0.00,0.00,0,0,0,0,0,10.65,26998
5024679,600295,145701000,23669995,10.49,10.49,0.00,1000,0,10.49,0.0,0.00,0.00,0.00,1000,10100,0,0,0,10.65,26998
5025394,600295,145703000,23669995,10.49,10.49,0.00,1900,0,10.49,0.0,0.00,0.00,0.00,1900,9200,0,0,0,10.65,26998


In [45]:
columns = ["StockID", "time", "cum_volume", "close", "bid1p", "bid2p", "bid1q", "bid2q",
           "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades"]
logSH[(logSH['StockID'] == 600295) & (logSH['time'] >= 145657000)][columns].head()

,StockID,time,cum_volume,close,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
13967681,600295,145657000,23669495,10.50,10.47,10.46,3000,4600,10.49,10.5,10.51,10.52,10.53,10400,19500,9500,28300,6800,10.65,26996
13968551,600295,145657970,23669995,10.49,10.47,10.46,3000,4600,10.49,10.5,10.51,10.52,10.53,10900,19500,9500,28300,6800,10.65,26998
13971615,600295,145700000,23669995,10.49,10.47,10.46,3000,4600,10.49,10.5,10.51,10.52,10.53,11100,19500,9500,28300,6800,10.65,26998
13971617,600295,145700000,23669995,10.49,0.00,0.00,0,0,0.00,0.0,0.00,0.00,0.00,0,0,0,0,0,10.65,26998
13972995,600295,145701540,23669995,10.49,10.49,0.00,1000,0,10.49,0.0,0.00,0.00,0.00,1000,10100,0,0,0,10.65,26998


In [19]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)
logSH1[(logSH1['StockID'] == 603535) & (logSH1['cum_volume'] >= 270100)][['StockID', 'time', 'cum_volume', 'close', 'bid1p', 'bid2p', 'ask1p', 'ask2p', 'bid1q', 'bid2q', 'ask1q', 'ask2q', 'numTrades']].head()

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q,numTrades
1028215,603535,100456000,270100,44.63,44.60,44.59,44.64,44.77,100,100,2200,1900,1747
1029315,603535,100459000,270200,44.64,44.60,44.59,44.64,44.77,100,100,2100,400,1748
1029316,603535,100459000,270200,44.64,44.60,44.59,44.64,44.77,100,100,2100,400,1748
1030372,603535,100502000,270300,44.64,44.61,44.60,44.64,44.77,100,100,2000,200,1749
1031615,603535,100504000,271000,44.64,44.61,44.60,44.64,44.78,100,100,1400,400,1754


In [20]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)
logSH[(logSH['StockID'] == 603535) & (logSH['cum_volume'] >= 270100) & (logSH['source'] == 23)][['StockID', 'time', 'cum_volume', 'close', 'bid1p', 'bid2p', 'ask1p', 'ask2p', 'bid1q', 'bid2q', 'ask1q', 'ask2q', 'numTrades']].head()

,StockID,time,cum_volume,close,bid1p,bid2p,ask1p,ask2p,bid1q,bid2q,ask1q,ask2q,numTrades
2631771,603535,100456000,270100,44.63,44.60,44.59,44.64,44.77,100,100,2200,1900,1747
2634182,603535,100459000,270100,44.63,44.60,44.59,44.64,44.77,100,100,2200,400,1747
2634183,603535,100459000,270200,44.64,44.60,44.59,44.64,44.77,100,100,2100,400,1748
2636618,603535,100502000,270300,44.64,44.61,44.60,44.64,44.77,100,100,2000,200,1749
2639558,603535,100504000,271000,44.64,44.61,44.60,44.64,44.78,100,100,1400,400,1754


In [3]:
import pandas as pd

path = np.array(glob.glob('/mnt/data/20201230/zt_7401_20201230_10.235.16.169_postvde.tar.gz/Logs_full/mdTradeLog_***'))
assert(len(path) == 1)
print(path[0])
path1 = np.array(glob.glob('/mnt/data/20201230/logs_20201230_zt_88_03_day_88data.tar.gz/mdTradeLog_***'))
assert(len(path1) == 1)
print(path1[0])

SH = pd.read_csv(path1[0], encoding="utf-8", index_col=False)
SH1 = pd.read_csv(path[0], encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
# ####
# SH = SH[SH['TransactTime'] <= SH1['TransactTime'].max()]
# ####

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']
# ####
# SH1 = SH1.drop_duplicates(columns, keep='first')
# ####


n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)


/mnt/data/20201230/zt_7401_20201230_10.235.16.169_postvde.tar.gz/Logs_full/mdTradeLog_20201230_0855.csv
/mnt/data/20201230/logs_20201230_zt_88_03_day_88data.tar.gz/mdTradeLog_20201230_0834.csv
1770
1457
{688128, 688129, 688133, 688135, 688138, 688139, 688155, 688156, 688157, 688158, 688159, 688160, 688165, 688166, 688168, 688169, 688177, 688178, 688179, 688180, 688181, 688185, 688186, 688188, 688189, 688196, 688198, 688199, 688200, 688202, 688208, 688215, 688218, 688219, 688221, 688222, 688228, 688229, 688233, 688256, 688258, 688266, 688268, 688277, 688278, 688286, 688288, 688289, 688298, 688299, 688300, 688301, 688308, 688309, 688310, 688311, 688312, 688313, 688318, 688321, 688330, 688333, 688335, 688336, 688338, 688339, 688356, 688357, 688358, 688360, 688363, 688365, 688366, 688368, 688369, 688377, 688378, 688379, 688386, 688388, 688389, 688390, 688393, 688396, 688398, 688399, 688408, 688418, 688466, 688488, 688500, 688505, 688508, 688513, 688516, 688518, 688519, 688520, 688521, 6885

In [50]:
SH1[SH1['SecurityID'] == 600023]['TransactTime'].min()

95207650